In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Rio Grande Do Sul - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [7]:
data = pd.read_csv('2003_mo_model_input_RS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH,Rio Grande Do Sul - Consumo de Cimento (t)
0,2003-1,8.192273,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,135.543840,2.066295e+08,8.907130e+06,18.206952,1.960625e+08,0.399576,0.775641,188.093
1,2003-2,8.186359,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,134.468148,2.068579e+08,8.912326e+06,18.211286,1.961183e+08,0.398811,0.775686,170.973
2,2003-3,8.180446,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,134.242963,2.070864e+08,8.917522e+06,18.215621,1.961742e+08,0.397898,0.775731,174.880
3,2003-4,8.174533,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,133.169391,2.073148e+08,8.922718e+06,18.219955,1.962300e+08,0.397042,0.775775,178.111
4,2003-5,8.168619,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,132.046874,2.075433e+08,8.927915e+06,18.224290,1.962858e+08,0.396050,0.775820,184.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.034199,NaN,NaN,NaN,NaN,0.832392,NaN,288.217
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.001080,NaN,NaN,NaN,NaN,0.829742,NaN,272.178
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.052788,NaN,NaN,NaN,NaN,0.826925,NaN,272.665
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.665535,NaN,NaN,NaN,NaN,0.823550,NaN,275.972


In [8]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
0,1.274013,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.577787,-1.677139,-1.973850,-2.280022,-2.069914,-1.362004,-1.691101
1,1.259068,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.629575,-1.660252,-1.934055,-2.253080,-2.045065,-1.371141,-1.675868
2,1.244123,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.640416,-1.643365,-1.894260,-2.226138,-2.020217,-1.382062,-1.660635
3,1.229178,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.692102,-1.626477,-1.854466,-2.199196,-1.995368,-1.392283,-1.645402
4,1.214233,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.746144,-1.609590,-1.814671,-2.172254,-1.970520,-1.404146,-1.630170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.966721,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.420132,1.083219,-0.768157,0.374323,0.568222,1.591021,0.807464
188,-0.973649,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.417824,1.070530,-0.760321,0.347418,0.545097,1.635348,0.773102
189,-0.980576,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.381727,1.057841,-0.752485,0.320513,0.521971,1.689944,0.738739
190,-0.987503,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.359803,1.045153,-0.744649,0.293608,0.498846,1.737741,0.704377


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      174.261
1      164.909
2      202.662
3      174.149
4      159.955
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Rio Grande Do Sul - Consumo de Cimento (t), Length: 240, dtype: float64

In [10]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
0,1.274013,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.577787,-1.677139,-1.973850,-2.280022,-2.069914,-1.362004,-1.691101
1,1.259068,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.629575,-1.660252,-1.934055,-2.253080,-2.045065,-1.371141,-1.675868
2,1.244123,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.640416,-1.643365,-1.894260,-2.226138,-2.020217,-1.382062,-1.660635
3,1.229178,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.692102,-1.626477,-1.854466,-2.199196,-1.995368,-1.392283,-1.645402
4,1.214233,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.746144,-1.609590,-1.814671,-2.172254,-1.970520,-1.404146,-1.630170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.092018,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.647118,1.197586,-0.541459,0.739492,0.925913,0.936850,1.430588
158,-0.148891,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.689363,1.198866,-0.569240,0.731546,0.918346,0.935032,1.419848
159,-0.205765,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.746341,1.200147,-0.597022,0.723600,0.910778,0.935266,1.409108
160,-0.262638,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.811207,1.201428,-0.624803,0.715654,0.903211,0.937596,1.398368


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      174.261
1      164.909
2      202.662
3      174.149
4      159.955
        ...   
157    196.675
158    251.395
159    199.081
160    220.793
161    211.546
Name: Rio Grande Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Rio Grande do Sul - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio Grande Do Sul - Produção de Cimento (t),Rio Grande do Sul - PIB - Estadual,Rio Grande do Sul - PIB - Construção Civil,Rio Grande do Sul - PIB - Per Capita,Rio Grande do Sul - PIB - Preços de Mercado,Rio Grande do Sul - value,Rio Grande do Sul - IDH
162,-0.376385,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-0.909322,1.203989,-0.680365,0.699762,0.888076,0.953539,1.376889
163,-0.433259,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-0.941615,1.205269,-0.708146,0.691816,0.880509,0.972469,1.366149
164,-0.490132,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-0.987957,1.206550,-0.735928,0.683871,0.872941,0.991650,1.355409
165,-0.547006,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-1.017508,1.207830,-0.763709,0.675925,0.865374,1.020849,1.344669
166,-0.603879,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-1.040475,1.209111,-0.791490,0.667979,0.857806,1.045546,1.333929
167,-0.660753,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-1.083059,1.210392,-0.819271,0.660033,0.850239,1.075359,1.323189
168,-0.717627,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-1.125192,1.211672,-0.847052,0.652087,0.842671,1.100640,1.312450
169,-0.734343,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-1.157418,1.208369,-0.845049,0.644635,0.833290,1.128617,1.290412
170,-0.751060,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-1.173773,1.205067,-0.843045,0.637182,0.823909,1.152032,1.268375
171,-0.767777,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-1.209404,1.201764,-0.841042,0.629730,0.814528,1.175722,1.246337


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    244.680
163    252.183
164    249.403
165    208.299
166    238.414
167    213.179
168    221.534
169    212.139
170    228.994
171    226.827
172    186.855
173    253.021
174    194.988
175    244.406
176    201.372
177    251.369
178    242.782
179    201.288
180    226.547
181    228.500
182    223.896
183    234.132
184    207.586
185    228.583
186    234.565
187    249.519
188    230.086
189    233.136
190    233.800
191    223.214
Name: Rio Grande Do Sul - Consumo de Cimento (t), dtype: float64

In [14]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 8)
#     target,target_val = validation_splitter(train_target, 8)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1790102656, 151355657, 3576083982, 1398084632, 4099399470, 9990582, 854507576, 1304042801, 2202375554, 1248764028, 2828668761, 1758614753, 403069063, 2056082736, 1692249364, 4102641171, 2546655372, 2627101091, 1925135277, 2397082750, 2729439295, 1078423881, 1561188886, 4269907804, 1988894673]


Step: 0 ___________________________________________
val_loss: 33443.1953125
winner_seed: 1790102656


Step: 1 ___________________________________________
val_loss: 35682.6015625


Step: 2 ___________________________________________
val_loss: 47971.71875


Step: 3 ___________________________________________
val_loss: 954.5722045898438
winner_seed: 1398084632


Step: 4 ___________________________________________
val_loss: 4154.68359375


Step: 5 ___________________________________________
val_loss: 66926.953125


Step: 6 ___________________________________________
val_loss: 31430.224609375


Step: 7 ___________________________________________


2023-10-14 17:59:51.966337: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 1418.941650390625


Step: 8 ___________________________________________
val_loss: 48369.0


Step: 9 ___________________________________________
val_loss: 24542.88671875


Step: 10 ___________________________________________
val_loss: 51664.375


Step: 11 ___________________________________________
val_loss: 42983.67578125


Step: 12 ___________________________________________
val_loss: 314.195068359375
winner_seed: 403069063


Step: 13 ___________________________________________
val_loss: 36009.46484375


Step: 14 ___________________________________________
val_loss: 13663.6796875


Step: 15 ___________________________________________
val_loss: 14244.6748046875


Step: 16 ___________________________________________
val_loss: 48368.65625


Step: 17 ___________________________________________
val_loss: 66097.296875


Step: 18 ___________________________________________
val_loss: 37000.72265625


Step: 19 ___________________________________________


2023-10-14 18:06:44.192480: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 73321.6953125


Step: 20 ___________________________________________
val_loss: 53892.4921875


Step: 21 ___________________________________________


2023-10-14 18:07:52.494423: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 30812.08203125


Step: 22 ___________________________________________
val_loss: 45299.1640625


Step: 23 ___________________________________________
val_loss: 3850.033203125


Step: 24 ___________________________________________
val_loss: 64862.41796875


final_seed: 403069063


In [19]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 22ms/step - loss: 57694.2344 - val_loss: 60628.8594
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 52362.3477 - val_loss: 35821.8477
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 45410.7578 - val_loss: 43446.7891
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 52019.3672 - val_loss: 3630.3198
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 38942.8398 - val_loss: 10932.9355
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 34312.6914 - val_loss: 21837.4004
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 30380.0957 - val_loss: 49817.4531
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 30372.2617 - val_loss: 2058.1643
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 27513.4336 - val_loss: 843.0748
Epoch 10/10000
5/5 [====================

5/5 [==============================] - 0s 5ms/step - loss: 266.0916 - val_loss: 1958.2817
Epoch 156/10000
5/5 [==============================] - 0s 5ms/step - loss: 262.2174 - val_loss: 1404.2802
Epoch 157/10000
5/5 [==============================] - 0s 5ms/step - loss: 296.9482 - val_loss: 1299.2500
Epoch 158/10000
5/5 [==============================] - 0s 5ms/step - loss: 297.5364 - val_loss: 1170.3308
Epoch 159/10000
5/5 [==============================] - 0s 5ms/step - loss: 292.0518 - val_loss: 1724.3141
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 271.2528 - val_loss: 1420.6588
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 282.2485 - val_loss: 1705.8157
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 256.2555 - val_loss: 2430.6985
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 275.5690 - val_loss: 1286.4933
Epoch 164/10000
5/5 [==============================] - 0s 4ms/

5/5 [==============================] - 0s 6ms/step - loss: 252.0975 - val_loss: 1607.2413
Epoch 310/10000
5/5 [==============================] - 0s 5ms/step - loss: 300.3772 - val_loss: 1553.9188
Epoch 311/10000
5/5 [==============================] - 0s 5ms/step - loss: 269.1984 - val_loss: 1275.1514
Epoch 312/10000
5/5 [==============================] - 0s 5ms/step - loss: 263.7887 - val_loss: 1309.1901
Epoch 313/10000
5/5 [==============================] - 0s 5ms/step - loss: 285.1822 - val_loss: 1228.4229
Epoch 314/10000
5/5 [==============================] - 0s 5ms/step - loss: 278.1650 - val_loss: 1445.5284
Epoch 315/10000
5/5 [==============================] - 0s 5ms/step - loss: 257.5631 - val_loss: 1382.7882
Epoch 316/10000
5/5 [==============================] - 0s 5ms/step - loss: 290.3789 - val_loss: 1388.2947
Epoch 317/10000
5/5 [==============================] - 0s 5ms/step - loss: 290.8592 - val_loss: 1449.0311
Epoch 318/10000
5/5 [==============================] - 0s 5ms/

5/5 [==============================] - 0s 5ms/step - loss: 217.1038 - val_loss: 1002.9897
Epoch 464/10000
5/5 [==============================] - 0s 5ms/step - loss: 221.5432 - val_loss: 1262.1476
Epoch 465/10000
5/5 [==============================] - 0s 5ms/step - loss: 221.1649 - val_loss: 959.2180
Epoch 466/10000
5/5 [==============================] - 0s 5ms/step - loss: 241.5123 - val_loss: 1375.0604
Epoch 467/10000
5/5 [==============================] - 0s 4ms/step - loss: 241.7386 - val_loss: 1572.9575
Epoch 468/10000
5/5 [==============================] - 0s 4ms/step - loss: 224.8563 - val_loss: 1215.7365
Epoch 469/10000
5/5 [==============================] - 0s 5ms/step - loss: 233.1152 - val_loss: 1602.3231
Epoch 470/10000
5/5 [==============================] - 0s 5ms/step - loss: 267.2530 - val_loss: 1285.7974
Epoch 471/10000
5/5 [==============================] - 0s 5ms/step - loss: 278.6769 - val_loss: 1199.3130
Epoch 472/10000
5/5 [==============================] - 0s 5ms/s

5/5 [==============================] - 0s 6ms/step - loss: 251.6989 - val_loss: 1223.1825
Epoch 618/10000
5/5 [==============================] - 0s 6ms/step - loss: 248.4798 - val_loss: 1071.2006
Epoch 619/10000
5/5 [==============================] - 0s 6ms/step - loss: 259.8005 - val_loss: 1373.8176
Epoch 620/10000
5/5 [==============================] - 0s 5ms/step - loss: 225.7286 - val_loss: 1255.0175
Epoch 621/10000
5/5 [==============================] - 0s 5ms/step - loss: 244.0316 - val_loss: 1489.7004
Epoch 622/10000
5/5 [==============================] - 0s 5ms/step - loss: 233.7098 - val_loss: 1079.4583
Epoch 623/10000
5/5 [==============================] - 0s 5ms/step - loss: 236.6557 - val_loss: 1177.8988
Epoch 624/10000
5/5 [==============================] - 0s 5ms/step - loss: 228.1266 - val_loss: 1143.3358
Epoch 625/10000
5/5 [==============================] - 0s 5ms/step - loss: 243.6392 - val_loss: 1249.9678
Epoch 626/10000
5/5 [==============================] - 0s 5ms/

5/5 [==============================] - 0s 5ms/step - loss: 227.8656 - val_loss: 901.8935
Epoch 772/10000
5/5 [==============================] - 0s 5ms/step - loss: 232.2110 - val_loss: 1193.4719
Epoch 773/10000
5/5 [==============================] - 0s 5ms/step - loss: 262.7018 - val_loss: 1075.1208
Epoch 774/10000
5/5 [==============================] - 0s 5ms/step - loss: 220.5192 - val_loss: 1130.8827
Epoch 775/10000
5/5 [==============================] - 0s 5ms/step - loss: 231.6620 - val_loss: 1136.2539
Epoch 776/10000
5/5 [==============================] - 0s 5ms/step - loss: 229.1158 - val_loss: 662.6960
Epoch 777/10000
5/5 [==============================] - 0s 5ms/step - loss: 235.0614 - val_loss: 1254.6074
Epoch 778/10000
5/5 [==============================] - 0s 5ms/step - loss: 231.8330 - val_loss: 834.9895
Epoch 779/10000
5/5 [==============================] - 0s 6ms/step - loss: 240.6155 - val_loss: 1241.8505
Epoch 780/10000
5/5 [==============================] - 0s 5ms/ste

5/5 [==============================] - 0s 5ms/step - loss: 215.1687 - val_loss: 1422.0089
Epoch 926/10000
5/5 [==============================] - 0s 5ms/step - loss: 217.1254 - val_loss: 1104.0159
Epoch 927/10000
5/5 [==============================] - 0s 5ms/step - loss: 229.4868 - val_loss: 1371.3263
Epoch 928/10000
5/5 [==============================] - 0s 5ms/step - loss: 254.2391 - val_loss: 1738.1672
Epoch 929/10000
5/5 [==============================] - 0s 5ms/step - loss: 259.2599 - val_loss: 1046.4808
Epoch 930/10000
5/5 [==============================] - 0s 5ms/step - loss: 234.2163 - val_loss: 1006.1827
Epoch 931/10000
5/5 [==============================] - 0s 6ms/step - loss: 220.9178 - val_loss: 984.8759
Epoch 932/10000
5/5 [==============================] - 0s 5ms/step - loss: 227.4394 - val_loss: 1153.0354
Epoch 933/10000
5/5 [==============================] - 0s 4ms/step - loss: 224.9665 - val_loss: 1383.8069
Epoch 934/10000
5/5 [==============================] - 0s 5ms/s

Epoch 1079/10000
5/5 [==============================] - 0s 5ms/step - loss: 224.8741 - val_loss: 1163.7994
Epoch 1080/10000
5/5 [==============================] - 0s 5ms/step - loss: 235.0398 - val_loss: 1434.4365
Epoch 1081/10000
5/5 [==============================] - 0s 5ms/step - loss: 228.1622 - val_loss: 901.4365
Epoch 1082/10000
5/5 [==============================] - 0s 5ms/step - loss: 230.5806 - val_loss: 1023.6575
Epoch 1083/10000
5/5 [==============================] - 0s 5ms/step - loss: 229.4783 - val_loss: 1079.3618
Epoch 1084/10000
5/5 [==============================] - 0s 5ms/step - loss: 232.3439 - val_loss: 1537.1134
Epoch 1085/10000
5/5 [==============================] - 0s 5ms/step - loss: 251.5629 - val_loss: 1196.1147
Epoch 1086/10000
5/5 [==============================] - 0s 5ms/step - loss: 227.7512 - val_loss: 828.7026
Epoch 1087/10000
5/5 [==============================] - 0s 5ms/step - loss: 226.6339 - val_loss: 861.4819
Epoch 1088/10000
5/5 [==================

In [20]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [21]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,215.792313,225.769333,215.916931,217.477066,216.576797,220.918655,221.44046,221.449417,221.456848,221.059464,215.87355,221.430389,219.539581,219.849106,215.967514,216.052292,219.472076,215.551147,210.588837,214.347183,209.921326,209.681305,209.662384,209.657181,209.65564,209.65744,209.655304,209.655243,209.655365,209.655197
Target,244.68,252.183,249.403,208.299,238.414,213.179,221.534,212.139,228.994,226.827,186.855,253.021,194.988,244.406,201.372,251.369,242.782,201.288,226.547,228.5,223.896,234.132,207.586,228.583,234.565,249.519,230.086,233.136,233.8,223.214
Error,28.88768,26.413666,33.486069,9.17807,21.837204,7.739655,0.093536,9.31041,7.537155,5.767532,29.018555,31.590607,24.551575,24.5569,14.59552,35.316711,23.309921,14.263153,15.95816,14.152817,13.97467,24.450699,2.076385,18.925812,24.909363,39.861557,20.430695,23.480759,24.144638,13.558807


In [22]:
display(mae)
display(mape)

19.445944

0.08431907

In [23]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [24]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2635.1614]] - Target[2735.5280000000002]| =  Error: [[100.3667]]; MAPE:[[0.03669006]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2570.29]] - Target[2685.449]| =  Error: [[115.158936]]; MAPE:[[0.04288256]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[1257.9342]] - Target[1404.32]| =  Error: [[146.38574]]; MAPE:[[0.1042396]]


[array([[100.3667]], dtype=float32),
 array([[115.158936]], dtype=float32),
 array([[146.38574]], dtype=float32)]

120.63712

0.061270744